<a href="https://colab.research.google.com/github/alibakhtiyari99/Learning-Git-start/blob/main/Reinforcment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import comtypes.client
import math
from sympy import symbols, Eq, solve
import pandas as pd

These are libraries that we need. `comtypes` for connecting to ETABS, `math` for Mathematical operations, `sympy` for Equations and `pandas` for export datas to excel

In [ ]:
try:
    myETABSObject = comtypes.client.GetActiveObject("CSI.ETABS.API.ETABSObject")
    SapModel = myETABSObject.SapModel
    print("✅ Connected to ETABS successfully.")
except Exception as e:
    print(f"❌ Failed to connect to ETABS: {e}")
    exit()


In this section we cintrol that python can connect to ETABS or not

In [ ]:
# Get table data
NumberTables = 1
TableKey = []
TableName = []
ImportType = []

# This shows all the available tables that can be accessed
ret = SapModel.DatabaseTables.GetAvailableTables(NumberTables,TableKey, TableName, ImportType)

TableKey = 'Concrete Beam Design Summary - ACI 318-19'
FieldKeyList = []

# set the group you want the results for, you can pick either 'All', 'Left Nodes', 'Right Nodes'
GroupName = ''
TableVersion = 1
FieldsKeysIncluded = []
NumberRecords = 1
TableData = []

BeamInf = SapModel.DatabaseTables.GetTableforDisplayArray(TableKey, FieldKeyList, GroupName, TableVersion, FieldsKeysIncluded, NumberRecords, TableData)

Now Based on the ETABS API documentation, we first use the `DatabaseTables.GetAvailableTables` class to retrieve all accessible tables and store them in `ret`. Then, in the next step, we specify the desired table and use the `DatabaseTables.GetTableForDisplayArray` class to extract the required data and store it in `BeamInf`.

In [ ]:
TableKey = 'Frame Section Property Definitions - Concrete Beam Reinforcing'
FieldKeyList = []


GroupName = ''
TableVersion = 1
FieldsKeysIncluded = []
NumberRecords = 1
TableData = []

CoverInf = SapModel.DatabaseTables.GetTableforDisplayArray(TableKey, FieldKeyList, GroupName, TableVersion, FieldsKeysIncluded, NumberRecords, TableData)
cover = int(CoverInf[4][3])

Like previous text we can use `DatabaseTables.GetTableforDisplayArray` to extract Beams Cover

In [ ]:
design=BeamInf[-2]
Rows=len(BeamInf[2])
DesignInf = tuple(design[i:i+Rows] for i in range(0, len(design), Rows))

Bot = {}
Top = {}
# Extract Each Beam Rebars
for item in DesignInf:
    group = item[1]  # Beam Label
    number = item[-4]  # As Bot
    number1 = item[-5]  # As Top

    # Making New List For New Beam
    if group not in Bot:
        Bot[group] = []

    Bot[group].append(number)

    if group not in Top:
        Top[group] = []

    Top[group].append(number1)

# Top Rebars & Bot Rebars In a List
Beams=[]
TopR=[]
BotR=[]
for group, numbers in Bot.items():
    Beams.append(group)
    BotR.append(numbers)

for group, numbers in Top.items():
    TopR.append(numbers)

#  Making Numbers Integer
TopRebar = [[int(x) for x in sublist] for sublist in TopR]
BotRebar = [[int(x) for x in sublist] for sublist in BotR]

At this stage, the required data stored in BeamInf, specifically array [-2], is first saved in design. Then, the information related to each row of the table is placed in a separate tuple to facilitate easier access.

Next, the data related to the required reinforcement areas for the top and bottom of each beam is stored. Finally, this data is converted into integers.

In [ ]:
# Longitudinal

RebarAs=[]
LongRebarAsTop=[]
for i in range(len(TopRebar)):

    if (math.ceil(len(TopRebar[i])/3)) % 2 !=0:
        Decision_SPoint=(math.ceil(len(TopRebar[i])/3))
    else:
        Decision_SPoint=(math.ceil(len(TopRebar[i])/3))-1

    Decision_EPoint=(len(TopRebar[i]))-(Decision_SPoint)
    LongRebarRange=TopRebar[i][Decision_SPoint:Decision_EPoint]
    LongRebar=max(LongRebarRange)
    LongRebarAsTop.append(LongRebar)
    OtherRebar1=TopRebar[i][0:Decision_SPoint]
    OtherRebar2=TopRebar[i][Decision_EPoint:]
    o1=max(OtherRebar1)
    o2=max(OtherRebar2)
    RebarAs.append([o1,o2])

After determining the reinforcement area for each beam, the next step is to specify the area for which the continuous reinforcement should be designed. To achieve this, based on the number of design stations for each beam, the maximum mid-span reinforcement area is selected as the required area for continuous reinforcement design.

In the following steps, if necessary, additional reinforcement is provided at the beam's start and end.

In [ ]:
# Number & Diameters
RebarList = [14 ,16 ,18 ,20 ,22 ,25 ,28 ,30 ,32]
User_d=input("Please Enter Your Custom Rebar Size For Top-Rebars(mm) (if You dont want, please enter ('All')) : ")

Report=[]
As=[]
if User_d == 'All' or User_d == 'all':
    for i in range (len(LongRebarAsTop)):
        for n in range (2,11,1) :
            x = symbols('x')
            eq = Eq(n*((math.pi * x**2)/4),LongRebarAsTop[i] )
            d1 = solve(eq, x)
            d1=d1[1]
            greater_numbers = [x for x in RebarList if x > d1]
            d = (min(greater_numbers) if greater_numbers else None)
            Scentre=(((width-2*cover)-(n*d))/(n-1))
            S=Scentre-d
            Smin=min(25,d)
            if S>Smin and S<200:
                Report.append(f"For {Beams[i]} Use {n}ϕ {d} & Rebar Free Spaces Are: {S}")
                As.append(n*(math.pi * d**2)/4)
                break
            else:
                continue

else:
    try:
        d = int(User_d)
        for i in range(len(LongRebarAsTop)):
            x = symbols('x')
            eq = Eq(x * ((math.pi * d**2) / 4), LongRebarAsTop[i])
            n = solve(eq, x)
            n = math.ceil(n[0])
            if n == 1:
                n = 2
            Scentre = (((width - 2 * cover) - (n * d)) / (n - 1))
            S=Scentre-d
            Smin = min(25, d)
            if S > Smin and S < 200:
                Report.append(f"For {Beams[i]} Use {n}ϕ {d} @ {S}")
                As.append(n * (math.pi * d**2) / 4)
            else:
                Report.append(f"❌Error in Design Code Space")
    except ValueError:
        print("Invalid input! Please enter a valid number or 'All'.")

In this section, the code asks the user whether they have a specific reinforcement diameter in mind or if they want to leave the selection to the predefined code logic. After that, by solving the required equation and calculating the number and diameter of the reinforcement bars, the outputs are stored in the Report.

In [ ]:
# Additional
ReportAdditionalTop = [[] for _ in range(len(Beams))]
for i in range (len(LongRebarAsTop)):
    RequierdAsLeft=RebarAs[i][0]-As[i]
    RequierdAsRight=RebarAs[i][1]-As[i]
    if RequierdAsLeft<=0:
        ReportAdditionalTop[i].append(f"You Dont Need Any Additional Rebar for Left of {Beams[i]} ")
    elif RequierdAsLeft<150 :
        ReportAdditionalTop[i].append("Please Increase Your Longitudinal Reinforcing")
    else :
        for n in range (1,11,1) :
            x = symbols('x')
            eq = Eq(n*((math.pi * x**2)/4),RequierdAsLeft)
            d1 = solve(eq, x)
            d1=d1[1]
            greater_numbers = [x for x in RebarList if x > d1]
            d = (min(greater_numbers) if greater_numbers else None)
            if n==1:
                ReportAdditionalTop[i].append(f"Add {n}ϕ {d} For Left {Beams[i]}")
            else:
                Scentre=(((width-2*cover)-(n*d))/(n-1))
                S=Scentre-d
                Smin=min(25,d)
                if S>Smin and S<200:
                    ReportAdditionalTop[i].append(f"Add {n}ϕ {d} For Left {Beams[i]}")
                    break
                else:
                    continue

    if RequierdAsRight<=0:
        ReportAdditionalTop[i].append(f"You Dont Need Any Additional Rebar for Right of {Beams[i]} ")
    elif RequierdAsRight<150 :
        ReportAdditionalTop[i].append("Please Increase Your Longitudinal Reinforcing")
    else :
        for n in range (1,11,1) :
            x = symbols('x')
            eq = Eq(n*((math.pi * x**2)/4),RequierdAsRight)
            d1 = solve(eq, x)
            d1=d1[1]
            greater_numbers = [x for x in RebarList if x > d1]
            d = (min(greater_numbers) if greater_numbers else None)
            if n==1:
                ReportAdditionalTop[i].append(f"Add {n}ϕ {d} For {Beams[i]}")
                break
            else:
                Scentre=(((width-2*cover)-(n*d))/(n-1))
                S=Scentre-d
                Smin=min(25,d)
                if S>Smin and S<200:
                    ReportAdditionalTop[i].append(f"Add {n}ϕ {d} For {Beams[i]}")
                    break
                else:
                    continue

At this stage, similar steps to the previous phase will be taken, with the key difference being that the required reinforcement area is now determined as the difference between the continuous reinforcement provided in the section and the required area at the beam's start and end. Finally, the outputs are stored in `ReportAdditionalTop`.

In [ ]:
file_path = "Reinforcing.xlsx"

df1 = pd.DataFrame(Report, columns=["Longitudinal Reinforcing"])
df2 = pd.DataFrame([" | ".join(map(str, row)) for row in ReportAdditionalTop], columns=["Additional Reinforcing"])

with pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:
    workbook = writer.book
    worksheet = workbook.add_worksheet("Reinforcing")
    writer.sheets["Reinforcing"] = worksheet

    header_format = workbook.add_format({
        "bold": True,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "#D7E4BC",
        "border": 1
    })


    worksheet.write(0, 0, "Longitudinal Reinforcing", header_format)
    worksheet.write(0, 1, "Additional Reinforcing", header_format)

    for row_idx, value in enumerate(df1["Longitudinal Reinforcing"], start=1):
        worksheet.write(row_idx, 0, value)

    for row_idx, value in enumerate(df2["Additional Reinforcing"], start=1):
        worksheet.write(row_idx, 1, value)

    worksheet.set_column("A:A", 50)
    worksheet.set_column("B:B", 100)

Finally, the data is saved in an Excel file using Pandas.